# An Exploratory Approach for Multi-perspective Event Abstraction

In [ ]:
import pandas as pd
from const import *
from clustering.retrieve import compute_similarities, retrieve_most_similar_events
from read import reader
from clustering import preprocessing
from clustering.clusterer import Clusterer
from config import Config
# the config object
config = Config("input/", "output/", "MPPNTaskAbstractionMobIS_pd_cases_fv_fine_1", clust="k_means")

# Get event representations from disk
pd_events_fv, loaded = reader.load_mppn_representations(config)



pca_df, pca_rep = preprocessing.pca(pd_events_fv)
print("PCA done")

preprocessing.viz(pca_df)



# Get similar events
#pd_events_fv = compute_similarities(pd_events_fv)
# Retrieve most similar events
#retrieve_most_similar_events(pd_events_fv, 5523, 6, top_n=5)

PCA done


In [2]:
clust = Clusterer(pd_events_fv, config)
clust.cluster(pca_rep, 27)
print("Clustering done")


0.40437767
Clustering done


In [3]:
# add the cluster labels to the dataframe
pd_events_fv[CLUST_COL] = clust.pred_labels

In [4]:
pd_events_fv


,IDX,case:concept:name,trace,fv,target_concept:name,target_concept:name_enc,event_pos,clust
0,0,1,case:concept:name concept:name case:travel...,"[4.1470294, 1.4067512, -4.3266125, 0.42475575,...",5.0,pay expenses,0,10
65,1,5,case:concept:name concept:name case:travel...,"[4.1470294, 1.4067512, -4.3266125, 0.42475575,...",5.0,pay expenses,0,10
130,2,6,case:concept:name concept:name case:...,"[5.0364475, -1.0454104, -2.2925222, -0.4669228...",6.0,calculate payments,0,10
131,2,6,case:concept:name concept:name case:...,"[-0.23424631, 0.6486099, -1.8590505, -0.101478...",5.0,pay expenses,1,10
195,3,7,case:concept:name co...,"[-4.600467, -2.7162364, 7.385008, -1.5052035, ...",7.0,send original documents to archive,0,22
...,...,...,...,...,...,...,...,...
425946,6553,7267,case:concept:name ...,"[-0.94584674, 1.85064, -1.78104, 2.039534, 1.1...",16.0,check if travel request needs preliminary pric...,1,0
425947,6553,7267,case:concept:name ...,"[-9.127682, 3.452376, 5.5292363, 0.12529077, 0...",17.0,decide on approval requirements,2,0
426010,6554,7268,case:concept:name ...,"[-2.1489437, 10.0971, -6.4524198, -0.61791253,...",15.0,file travel request,0,13
426011,6554,7268,case:concept:name ...,"[-1.1794254, 2.0246515, -1.8443458, 2.0812948,...",16.0,check if travel request needs preliminary pric...,1,0


In [5]:
# get events of a cluster and compute properties
clust_to_props = {}
for clust, events in pd_events_fv.groupby(CLUST_COL):
    event_types = set()
    resources = set()
    roles = set()
    cat_atts = dict()
    num_atts = dict()
    preceded_by = set()
    followed_by = set()
    min_dur_per_case = 0
    max_dur_per_case = 0
    avg_dur_per_case = 0

    all_group_durs = set()
    for case, events_per_case in events.groupby(XES_CASE):
        #print(case)
        indices = events_per_case[EVENT_POS_IN_CASE].tolist()
        #print(indices)
        if len(events_per_case) < 1:
            continue
        case_full = events_per_case.iloc[0][TRACE_DF].iloc[indices]
        # else:
        #    case_full = events_per_case[TRACE_DF]
        #len(case_full)
        min_time = case_full[XES_TIME].min()
        max_time = case_full[XES_TIME].max()
        all_group_durs.add(max_time - min_time)
        # add event / activity types
        event_types.update(case_full[XES_NAME].tolist())
        # add resources
        resources.update(case_full[XES_RESOURCE].tolist())
        # add role
        roles.update(case_full[MOBIS_ROLE].tolist())
    for row in events.itertuples():
        try:
            preceded_by.add(row[COL_INDEX_TRACE].iloc[row[COL_INDEX_POS] - 1][XES_NAME])
        except:
            pass
        try:
            followed_by.add(row[COL_INDEX_TRACE].iloc[row[COL_INDEX_POS] + 1][XES_NAME])
        except:
            pass
    #preceded_by = preceded_by.difference(event_types)
    #followed_by = followed_by.difference(event_types)
    #print("resources", resources)
    #print("durations", all_group_durs)
    #print("roles", roles)
    print("activities", event_types)
    #print("predecessors", preceded_by)
    #print("followers", followed_by)


activities {'file travel request', 'decide on travel expense approval', 'check if expense documents exist', 'decide on approval requirements', 'check if price inquiry is still needed and up to date', 'upload travel expense documents', 'send booking proposal to employee', 'pay expenses', 'decide on request', 'calculate payments', 'correct request', 'check if travel request is still needed and up to date', 'book travel', 'file travel expense report', 'send request for correction', 'transform price inquiry to travel request', 'check booking proposal', 'check if travel request needs preliminary price inquiry', 'send original documents to archive', 'send request for travel expense correction', 'request update of the booking proposal', 'forward request to approver', 'correct travel expense report', 'prepare booking proposal', 'check if booking is necessary'}
activities {'book travel', 'check if price inquiry is still needed and up to date', 'check if travel request needs preliminary price in

NameError: name 'np' is not defined